# 🧩 Ajustes Pós-Clusterização com LLM

Este notebook realiza a **etapa de refinamento e interpretação dos clusters** obtidos na modelagem de tópicos (BERTopic).  
Ele aplica **modelos de linguagem (LLMs)** para:

- Gerar **rótulos automáticos** para cada cluster, considerando os documentos mais representativos.  
- **Ajustar e unificar clusters semelhantes** com base em similaridade semântica e validação pelo LLM.  
- Atualizar os registros no banco de dados (`processos_rag`) com novas colunas descritivas (e.g., `descricao_topico_llm`, `numero_topico_llm`, `problema_central_agrupado`).  
- Criar uma **camada de interpretação final**, conectando tópicos jurídicos semelhantes e consolidando respostas extraídas dos processos.

> 📊 Etapa essencial do pipeline **JuriDiscovery**, responsável por transformar os resultados brutos da clusterização em insights jurídicos compreensíveis e acionáveis.


In [1]:

import os
import re
import time
import json
import math
import warnings
from datetime import datetime, date
from collections import Counter
from psycopg2.extras import execute_values
from typing import Optional
import requests
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Circle
from adjustText import adjust_text
from dotenv import load_dotenv
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from IPython.display import display, clear_output
from collections import defaultdict, deque

warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy.*")


In [2]:
# Conexão
def get_connection():
    return psycopg2.connect(
        dbname="PROCESSOS",
        user="postgres",
        password="",
        host="",
        port="5432"
    )

In [3]:
# Variáveis globais para armazenar o token e o horário em que foi gerado
token = None
last_token_time = 0

# Função para obter o token da API
def get_token():
    client_id = ''
    client_secret = ''
    result = requests.request('POST', 
        "...", 
        data={"grant_type":"client_credentials"}, 
        auth=(client_id, client_secret))

    if result.ok:
        return result.json()['access_token']
    else:
        raise Exception("Erro ao obter o token")

# Função para verificar se o token está expirado (renova se necessário)
def get_valid_token():
    global token, last_token_time
    current_time = time.time()
    
    # Se o token não existe ou já passaram mais de 20 minutos, obtemos um novo
    if token is None or (current_time - last_token_time) > 20 * 60:  # 20 minutos em segundos
        token = get_token()
        last_token_time = current_time
        print("Novo token obtido.")

    return token

# Função para invocar o LLM
def invoke(prompt, modelo,token, temperature=0, max_tokens=10000, stream=False):
    # Obtém o token válido (renova se necessário)
    

    payload_data = {
        "model": modelo,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": stream
    }

    result = requests.request("POST", 
        '...', 
        

        data=json.dumps(payload_data), 
        headers={
            "Authorization": f"Bearer {token}", 
            "Content-Type":"application/json"})

    if result.ok:
        res = result.json()
        resposta = res["choices"][0]["message"]['content']
        return resposta
    else:
        print(result.text)


In [4]:
def gerar_e_salvar_clusters(modelo, token):
    conn = get_connection()
    cur = conn.cursor()
    modelo = 'pixtral-12b'

    # =========================
    # 0) Limpar campos antigos antes de atualizar
    # =========================
    print("🧹 Limpando campos anteriores dos clusters...")
    cur.execute("""
        UPDATE processos SET
            descricao_curta_cluster = NULL,
            descricao_longa_cluster = NULL,
            questoes_discussao_cluster = NULL,
            solucoes_propostas_cluster = NULL,
            teses_cluster = NULL
    """)
    conn.commit()
    print("✅ Campos limpos com sucesso.\n")

    df = pd.read_sql_query("""
        SELECT id, numero_processo_tribunal, descricao_caso, questoes_em_discussao, 
               solucoes_propostas, tese, numero_topico_llm, descricao_topico_llm, 
               proximo_do_centroid
        FROM processos 
        WHERE proximo_do_centroid = 1
    """, conn)

    grupos = df.groupby("descricao_topico_llm")

    for topico, grupo in grupos:
        casos = "\n\n".join(grupo['descricao_caso'].dropna())

        questoes = "\n\n".join(grupo['questoes_em_discussao'].dropna())
        solucoes = "\n\n".join(grupo['solucoes_propostas'].dropna())
        teses = "\n\n".join(grupo['tese'].dropna())

        clear_output(wait=True)
        print(f"\n\n===========================")
        print(f"🔎 Tópico {topico} - {grupo['descricao_topico_llm'].iloc[0]}")
        print(f"IDs: {grupo['id'].tolist()}")

        # 1. Descrição curta
        prompt_1 = "Você é um assistente jurídico. Com base nas descrições dos casos abaixo, escreva um resumo curto (1 parágrafo com até 150 caracteres) que represente o tema comum entre os processos:\n\n" + casos
        texto_1 = invoke(prompt_1, modelo, token)
        print("\n📌 Descrição curta:")
        print(texto_1)

        
        # Atualiza apenas uma vez por topico LLM
        cur.execute("""
            UPDATE processos SET 
                descricao_curta_cluster = %s
            WHERE descricao_topico_llm = %s
        """, (
            texto_1
        ))
        conn.commit()

    cur.close()
    conn.close()


In [5]:
gerar_e_salvar_clusters("pixtral-12b", get_token())



🔎 Tópico Vínculo Empregatício - Vínculo Empregatício
IDs: [31896, 8765, 13719, 5698, 14611, 28501, 25338, 14344, 42309, 36691, 11256, 35235, 8396, 43466, 16946, 35076, 44081, 10614, 16731, 35651, 36452, 42956, 10501, 6132, 6178, 35582, 8433, 30897, 43659, 14912, 35077, 8373, 12491, 42650, 6454, 9463, 12902, 15219, 17141, 8525, 26250, 43173, 11443, 9139, 23714, 44397, 7100, 25271, 6406, 7179, 10769, 13210, 45160, 8247, 5985, 9333, 7386, 5750, 9191, 36301, 44499, 10750, 23467, 24434, 27675, 24780, 36541, 16660, 23204, 6630, 26108, 7099, 7976, 11210, 24618, 26113, 25583, 5967, 7385, 42775, 36312, 12463, 15885, 23838, 18253, 44825, 14089, 36045, 35858, 44500, 6415, 8679, 7826, 17971, 14323, 27593, 42955, 7895, 43748, 27246, 16360, 27364, 8313, 28298, 27363, 8257, 27188, 8274, 8384, 18598, 43903, 35459, 8301, 36413, 28085, 14970, 17686, 27824, 18046, 36388, 24832, 5973, 27030, 12327, 4808, 26498, 43025, 11209, 32061, 42615, 45020, 30349, 7259, 16386, 24737, 23943, 17374, 36290, 35377, 367

## Agrupando os tópicos
### 1 - Gera o embedding do nome do tópico + descrição curta (isso foi testado em várias estrategias diferentes)

### 2 - Cria uma matriz de similaridade de cada par de topicos onde o valor da célula é a similaridade entr5e os topicos

### 3 - Pergunta ao LLM, para aqueles pares com similaridade maior que um limiar, se eles poderiam ser agrupados

### 4 - Gerra a matriz de similiaridade de cada par de topicos onde o valor da célula é 1, caso os topicos sejam semelhantes ou 0 caso não sejam

In [6]:
# =========================
# PARÂMETROS AJUSTÁVEIS
# =========================
DB_NAME = "PROCESSOS"
DB_USER = ""
DB_PASS = ""
DB_HOST = ""
DB_PORT = "5432"

BERT_LOCAL_PATH = "C:/...t"  # caminho local do BERTimbau
MAX_LENGTH = 512
LIMIAR_MUITO_PARECIDO = 0.80     # limiar para enviar par ao LLM

CSV_SIMILARIDADE = "matriz_similaridade_topicos.csv"
CSV_BINARIA = "matriz_mesmo_problema_llm.csv"

LLM_MODELO = "pixtral-12b"
LLM_DELAY_S = 0.05               # atraso entre chamadas
CASAS_PRINT = 3                  # casas decimais para imprimir similaridade

# =========================
# 0) Autenticação e chamada ao LLM (SERPRO)
# =========================
token = None
last_token_time = 0.0

def get_token():
    client_id = ''
    client_secret = ''
    result = requests.request(
        'POST',
        "...",
        data={"grant_type": "client_credentials"},
        auth=(client_id, client_secret),
        timeout=60
    )
    if result.ok:
        return result.json()['access_token']
    else:
        raise Exception(f"Erro ao obter o token: {result.text}")

def get_valid_token():
    global token, last_token_time
    current_time = time.time()
    if token is None or (current_time - last_token_time) > 20 * 60:  # 20 minutos
        token = get_token()
        last_token_time = current_time
        print("Novo token obtido.")
    return token

def invoke(prompt, modelo, token, temperature=0, max_tokens=8, stream=False):
    payload_data = {
        "model": modelo,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stream": stream
    }
    result = requests.request(
        "POST",
        "...",
        data=json.dumps(payload_data),
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        timeout=120
    )
    if result.ok:
        res = result.json()
        return res["choices"][0]["message"]["content"]
    else:
        raise RuntimeError(f"Erro no LLM: {result.status_code} - {result.text}")

# =========================
# 1) Conexão e busca de tópicos
# =========================
def conectar_banco():
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    return conn

def buscar_topicos(conn):
    sql = """
        SELECT
            descricao_topico_llm,
            descricao_curta_cluster,
            questoes_discussao_cluster,
            cor_cluster,
            COUNT(*) AS total_registros
        FROM processos
        WHERE descricao_topico_llm IS NOT NULL
          AND descricao_curta_cluster IS NOT NULL
        GROUP BY
            descricao_topico_llm,
            descricao_curta_cluster,
            questoes_discussao_cluster,
            cor_cluster
        ORDER BY total_registros DESC
    """
    df = pd.read_sql_query(sql, conn)
    return df.reset_index(drop=True)

# =========================
# 2) Preparar texto de cada tópico
# =========================
def _safe_str(x):
    return "" if x is None else str(x)

def montar_texto_topico(row):
    """
    Concatena: descricao_topico_llm [SEP] descricao_curta_cluster [SEP] questoes_discussao_cluster
    (Atualmente está usando apenas o título para embedding, conforme seu ajuste.)
    """
    p1 = _safe_str(row.get("descricao_topico_llm", ""))
    # Para usar também as outras descrições, descomente:
    #p2 = _safe_str(row.get("descricao_curta_cluster", ""))
    # p3 = _safe_str(row.get("questoes_discussao_cluster", ""))
    p3 = p2 = ""
    def _clean(s): return " ".join(s.split()).strip()
    return " [SEP] ".join([_clean(p1), _clean(p2), _clean(p3)])

# =========================
# 3) Vetorização com BERTimbau local (robusta a overflow)
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(BERT_LOCAL_PATH)
model = AutoModel.from_pretrained(BERT_LOCAL_PATH).to(device)
model.eval()

def gerar_vetor(texto: str, max_length: int = MAX_LENGTH) -> np.ndarray:
    """
    Embedding por mean pooling com máscara (ignora padding) + normalização L2.
    Robusto a estouro de tokens (alinha máscara ao tamanho real de saída do modelo).
    Retorna np.ndarray shape (hidden_size,).
    """
    if not isinstance(texto, str):
        texto = "" if texto is None else str(texto)

    # Respeita limites do modelo/tokenizer
    model_max = int(getattr(model.config, "max_position_embeddings", 512))
    tok_max = getattr(tokenizer, "model_max_length", model_max)
    if tok_max is None or tok_max > 10_000:
        tok_max = model_max
    effective_max = int(min(max_length, model_max, tok_max))
    effective_max = max(8, effective_max)

    with torch.no_grad():
        ins = tokenizer(
            texto,
            return_tensors="pt",
            truncation=True,
            padding=False,
            max_length=effective_max
        ).to(device)

        out = model(**ins)
        hidden = out.last_hidden_state              # [1, Tm, H]
        seq_len = hidden.size(1)                    # Tm
        attn = ins["attention_mask"][:, :seq_len]   # [1, Tm]
        mask = attn.unsqueeze(-1).expand(hidden.size()).float()

        sent = (hidden * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)   # [1, H]
        sent = F.normalize(sent, p=2, dim=1)                                   # [1, H]
        return sent.squeeze(0).detach().cpu().numpy().astype(np.float32)

def embed_textos(textos):
    """Gera um vetor por texto chamando gerar_vetor()."""
    vecs = [gerar_vetor(t) for t in tqdm(textos, desc="Gerando embeddings")]
    if not vecs:
        dim = getattr(model.config, "hidden_size", 768)
        return np.empty((0, dim), dtype=np.float32)
    return np.vstack(vecs)

# =========================
# 4) Matriz de similaridade
# =========================
def matriz_similaridade(embs: np.ndarray) -> np.ndarray:
    """
    Como os embeddings já estão normalizados (L2), o produto interno vira cosseno.
    Retorna matriz S (N x N) com diag=1.0.
    """
    if embs.size == 0:
        return np.empty((0, 0), dtype=np.float32)
    S = embs @ embs.T
    np.fill_diagonal(S, 1.0)
    return S.astype(np.float32)

# =========================
# 5) Impressão/CSV com rótulos
# =========================
def imprimir_dataframe_com_rotulos(matriz, labels, casas=3, titulo=None):
    """
    Imprime um DataFrame com rótulos nas linhas e colunas.
    """
    if titulo:
        print(f"\n=== {titulo} ===")
    df = pd.DataFrame(np.round(matriz, casas), index=labels, columns=labels)
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 200)
    df

def salvar_csv_com_rotulos(matriz, labels, path_csv):
    """
    Salva CSV com rótulos de linhas e colunas.
    """
    df_full = pd.DataFrame(matriz, index=labels, columns=labels)
    df_full.to_csv(path_csv, encoding="utf-8", index=True)
    print(f"[OK] CSV salvo: {path_csv}")

# =========================
# 6) Similaridade: pipeline + impressão
# =========================
def imprimir_matriz_similaridade(conn, arredondar=3, salvar_csv=CSV_SIMILARIDADE):
    # 6.1 Buscar dados
    df_top = buscar_topicos(conn)
    if df_top.empty:
        print("Nenhum tópico encontrado.")
        return df_top, np.empty((0, 0)), pd.DataFrame()

    # 6.2 Texto completo por tópico
    df_top["texto_full"] = df_top.apply(montar_texto_topico, axis=1)

    # 6.3 Embeddings
    embs = embed_textos(df_top["texto_full"].tolist())

    # 6.4 Matriz de similaridade
    S = matriz_similaridade(embs)

    # 6.5 Rótulos únicos (duplicados desambiguados)
    labels = df_top["descricao_topico_llm"].astype(str).tolist()
    count = Counter(labels)
    seen = defaultdict(int)
    labels_uniq = []
    for L in labels:
        if count[L] > 1:
            seen[L] += 1
            labels_uniq.append(f"{L} [{seen[L]}]")
        else:
            labels_uniq.append(L)

    # 6.6 Imprimir e salvar
    imprimir_dataframe_com_rotulos(S, labels_uniq, casas=arredondar, titulo="MATRIZ DE SIMILARIDADE (cos)")
    if salvar_csv:
        salvar_csv_com_rotulos(S, labels_uniq, salvar_csv)

    # Retorna DataFrame com rótulos nas colunas/índice
    sim_df = pd.DataFrame(S, index=labels_uniq, columns=labels_uniq)
    return df_top, embs, sim_df

# =========================
# 7) LLM: prompt e normalização 1/0
# =========================
def construir_prompt_mesmo_problema(topico_a, desc_a, quest_a, topico_b, desc_b, quest_b):
    qa = (quest_a or "").strip()
    qb = (quest_b or "").strip()
    return f"""
Você é um assistente jurídico. Decida se os dois tópicos abaixo tratam essencialmente do MESMO PROBLEMA central,
desconsiderando variações de redação. Responda APENAS com um único caractere: "1" (mesmo problema) ou "0" (problemas diferentes).

Tópico A:
- Título: {topico_a}
- Descrição curta: {desc_a}

Tópico B:
- Título: {topico_b}
- Descrição curta: {desc_b}

Responda somente com 1 ou 0:
""".strip()

def normalizar_0_1(texto) -> int:
    if not isinstance(texto, str):
        return 0
    t = texto.strip()
    if t.startswith("1"): return 1
    if t.startswith("0"): return 0
    return 0

# =========================
# 8) Matriz binária NxN via LLM (consome df_top, sim_df)
# =========================
def construir_matriz_binaria_por_llm(
    df_top: pd.DataFrame,
    sim_df: pd.DataFrame,
    limiar: float = LIMIAR_MUITO_PARECIDO,
    modelo: str = LLM_MODELO,
    delay_s: float = LLM_DELAY_S,
    casas_print: int = 0,
    csv_saida: str = CSV_BINARIA
):
    """
    Constrói a matriz binária NxN (1 = mesmo problema; 0 = diferente) usando o LLM,
    a partir de df_top e da matriz de similaridade sim_df retornados por imprimir_matriz_similaridade.
    Imprime a matriz como DataFrame com nomes nas linhas e colunas e salva CSV.

    Retorna:
      - M (np.ndarray NxN)
      - labels (lista de rótulos)
      - df_bin (DataFrame NxN com rótulos)
    """
    # 8.1 Extrai similaridade e rótulos na MESMA ORDEM
    S = sim_df.values.astype(float)
    labels = list(sim_df.index)  # ordem já consistente entre índice e colunas
    n = S.shape[0]

    if n == 0:
        print("Matriz de similaridade vazia. Nada a validar no LLM.")
        return np.empty((0, 0), dtype=np.int8), [], pd.DataFrame()

    # 8.2 Preparar descrições/questões para prompts
    topicos_df = df_top["descricao_topico_llm"].astype(str).tolist()
    descs_df   = df_top["descricao_curta_cluster"].astype(str).tolist()
    quests_df  = df_top.get("questoes_discussao_cluster", pd.Series([""] * len(df_top))).astype(str).tolist()

    # mapa rápido por título → (desc, quest)
    mapa_por_titulo = {}
    for t, d, q in zip(topicos_df, descs_df, quests_df):
        if t not in mapa_por_titulo:
            mapa_por_titulo[t] = (d, q)

    def obter_desc_e_quest_por_pos(i):
        titulo = labels[i]
        if titulo in mapa_por_titulo:
            return mapa_por_titulo[titulo]
        # fallback pela posição (assumindo mesma execução/ordem)
        if i < len(descs_df):
            return descs_df[i], quests_df[i]
        return "", ""

    # 8.3 Matriz 1/0
    M = np.zeros((n, n), dtype=np.int8)
    np.fill_diagonal(M, 1)

    print(f"\n[LLM] Avaliando pares com similaridade >= {limiar:.3f} (resposta: 1/0)...")
    for i in range(n):
        di, qi = obter_desc_e_quest_por_pos(i)
        for j in range(i + 1, n):
            if S[i, j] >= limiar:
                dj, qj = obter_desc_e_quest_por_pos(j)
                prompt = construir_prompt_mesmo_problema(
                    labels[i], di, qi,
                    labels[j], dj, qj
                )
                # Logs opcionais (tire se quiser rodar silencioso)
                clear_output(wait=True)
                print(f"[{i},{j}] {labels[i]}  ↔  {labels[j]}\n")
                print(prompt)

                try:
                    tok = get_valid_token()
                    resp = invoke(prompt, modelo, tok, temperature=0, max_tokens=8, stream=False)
                    #bit = normalizar_0_1(resp)
                    bit = resp
                    print(f"LLM → {resp!r}  ⇒  {bit}")
                except Exception as e:
                    print(f"[WARN] Erro LLM no par ({i},{j}) '{labels[i]}' vs '{labels[j]}': {e}")
                    bit = 0

                M[i, j] = M[j, i] = bit
                time.sleep(delay_s)
            else:
                M[i, j] = M[j, i] = 0

    # 8.4 Salvar e imprimir com rótulos
    if csv_saida:
        salvar_csv_com_rotulos(M, labels, csv_saida)
    imprimir_dataframe_com_rotulos(M, labels, casas=casas_print, titulo="MATRIZ MESMO PROBLEMA (LLM) — 1/0")

    df_bin = pd.DataFrame(M, index=labels, columns=labels)
    return M, labels, df_bin




Some weights of the model checkpoint at C:/Users/Loreane/Documents/bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
conn = conectar_banco()
# 1) Similaridade
df_top, embs, sim_df = imprimir_matriz_similaridade(
            conn,
            arredondar=CASAS_PRINT,
            salvar_csv=CSV_SIMILARIDADE
        )
sim_df

Gerando embeddings:   0%|          | 0/55 [00:00<?, ?it/s]


=== MATRIZ DE SIMILARIDADE (cos) ===
[OK] CSV salvo: matriz_similaridade_topicos.csv


,Vínculo Empregatício,Justiça Gratuita,Isenção de Taxas para MEIs,Fraude Empresarial,Adeção ao Simples Nacional,Seguro-Desemprego e Renda,Problemas Bancários,Confusão Patrimonial,Negativação Indevida,Contribuições e Aposentadoria,Benefícios Previdenciários,Irregularidades em Licitações,Recursos Trabalhistas,Redução de Depósito Recursal,Salário-Maternidade INSS,Juros e Cobrança,Documentação Judicial,Auxílio Emergencial,Cobrança de Débitos,Problemas com Planos de Saúde,Benefícios para Deficientes,Liberdade Provisória,Pensão por Morte,Cobrança de Dívidas Escolares,Licenciamento MEI,Legitimidade Judicial,Registro Veterinário,Cobrança de ICMS DIFAL,Competência Juizados Especiais,Acidentes de Trânsito,Classificação de Segurados,Problemas com Operadoras de Telefonia,Impenhorabilidade de Bens Profissionais,Legitimidade em Execuções,Trabalho Externo em Regime Semiaberto,Enquadramento Judicial,Redução de Alimentos,Legitimidade nos Juizados Especiais,Execução de Títulos,Prisão em Processos,Impenhorabilidade de Valores,Competência Juizado Especial,Pagamentos de Serviços,Cobrança sem Nota Fiscal,Legitimidade no Juizado Especial,Benefícios Assistenciais,Aposentadoria Rural,Gratuidade Financeira,Cobrança Indevida,Dívida e Conciliação,Citação Válida,Embargos de Declaração,Crimes e Provas,Justiça Gratuita para Recursos,Doações Eleitorais
Vínculo Empregatício,1.000000,0.638609,0.643226,0.671373,0.647638,0.689192,0.638261,0.694010,0.620028,0.642942,0.654179,0.577377,0.595478,0.626730,0.653327,0.611700,0.631329,0.689534,0.624202,0.593055,0.614962,0.604954,0.598648,0.578162,0.660743,0.547658,0.594628,0.636662,0.539177,0.589791,0.615942,0.560619,0.662521,0.545682,0.726247,0.710407,0.551878,0.513318,0.600766,0.592161,0.688525,0.548411,0.604846,0.594579,0.543885,0.655932,0.664163,0.645454,0.645719,0.619460,0.639932,0.471425,0.576065,0.604939,0.562687
Justiça Gratuita,0.638609,1.000000,0.692620,0.672157,0.726324,0.649667,0.620832,0.682775,0.673504,0.639665,0.643831,0.604799,0.714376,0.637236,0.616275,0.708844,0.746387,0.716200,0.647965,0.597866,0.691628,0.790582,0.713321,0.605704,0.710070,0.648630,0.585681,0.656852,0.657955,0.581118,0.672934,0.571887,0.611088,0.658487,0.681339,0.780266,0.629805,0.624067,0.678468,0.735530,0.628318,0.648513,0.689674,0.669875,0.636854,0.695672,0.708705,0.809478,0.734014,0.698544,0.726261,0.530149,0.685359,0.938233,0.649253
Isenção de Taxas para MEIs,0.643226,0.692620,1.000000,0.681698,0.784936,0.720082,0.691940,0.645308,0.637590,0.719264,0.744213,0.703822,0.668238,0.689031,0.643081,0.834351,0.627534,0.667187,0.779729,0.689769,0.683089,0.632443,0.604116,0.776477,0.759853,0.539352,0.586546,0.838754,0.571948,0.628946,0.660836,0.656315,0.693569,0.591617,0.686748,0.699354,0.638737,0.548492,0.675759,0.639504,0.662964,0.540918,0.746764,0.812157,0.536938,0.699066,0.671801,0.728433,0.759174,0.664669,0.633218,0.538824,0.639393,0.702807,0.629741
Fraude Empresarial,0.671373,0.672157,0.681698,1.000000,0.683980,0.626422,0.716425,0.826276,0.715207,0.638614,0.591865,0.695042,0.688947,0.639261,0.608370,0.707196,0.663663,0.640293,0.702079,0.670596,0.600273,0.655752,0.671573,0.677633,0.695217,0.637190,0.598376,0.668326,0.568507,0.671590,0.679544,0.671900,0.709002,0.668033,0.642397,0.713995,0.635156,0.563586,0.673405,0.707395,0.723077,0.572912,0.669429,0.712828,0.577681,0.599115,0.681763,0.681299,0.802574,0.723394,0.701933,0.539381,0.686348,0.642894,0.583348
Adeção ao Simples Nacional,0.647638,0.726324,0.784936,0.683980,1.000000,0.703580,0.614167,0.666272,0.637605,0.698340,0.694317,0.627922,0.676797,0.624891,0.658208,0.716673,0.691482,0.654071,0.690739,0.619499,0.645934,0.649005,0.651002,0.658863,0.725394,0.607563,0.626404,0.737914,0.544564,0.582428,0.662095,0.570873,0.614974,0.621589,0.665711,0.765217,0.603372,0.560967,0.606028,0.631332,0.614252,0.545896,0.640564,0.768897,0.579725,0.628286,0.726034,0.692418,0.714020,0.688523,0.635828,0.533949,0.615530,0.698626,0.594523
Seguro-Desemprego e Renda,0.689192,0.649667,0.720082,0.626422,0.703580,

In [8]:
    # 2) Matriz binária via LLM (executar após a similaridade)
M, labels, df_bin = construir_matriz_binaria_por_llm(
        df_top=df_top,
        sim_df=sim_df,
        limiar=0.75,
        modelo=LLM_MODELO,
        delay_s=LLM_DELAY_S,
        casas_print=0,
        csv_saida=CSV_BINARIA
    )

df_bin

[48,50] Cobrança Indevida  ↔  Citação Válida

Você é um assistente jurídico. Decida se os dois tópicos abaixo tratam essencialmente do MESMO PROBLEMA central,
desconsiderando variações de redação. Responda APENAS com um único caractere: "1" (mesmo problema) ou "0" (problemas diferentes).

Tópico A:
- Título: Cobrança Indevida
- Descrição curta: Os casos envolvem MEIs que entraram com ações contra municípios alegando cobrança indevida de taxas não previstas em lei. Em alguns casos, o município contestou, argumentando a legalidade e proporcionalidade das cobranças. O tema comum é a contestação de cobranças administrativas consideradas indevidas por MEIs.

Tópico B:
- Título: Citação Válida
- Descrição curta: Os casos apresentados giram em torno de questionamentos sobre a validade da citação processual, seja por problemas de endereço, ausência de identificação do recebedor, recebimento por terceiros ou outras irregularidades, impactando a regularidade dos processos e a eficácia das notifi

,Vínculo Empregatício,Justiça Gratuita,Isenção de Taxas para MEIs,Fraude Empresarial,Adeção ao Simples Nacional,Seguro-Desemprego e Renda,Problemas Bancários,Confusão Patrimonial,Negativação Indevida,Contribuições e Aposentadoria,Benefícios Previdenciários,Irregularidades em Licitações,Recursos Trabalhistas,Redução de Depósito Recursal,Salário-Maternidade INSS,Juros e Cobrança,Documentação Judicial,Auxílio Emergencial,Cobrança de Débitos,Problemas com Planos de Saúde,Benefícios para Deficientes,Liberdade Provisória,Pensão por Morte,Cobrança de Dívidas Escolares,Licenciamento MEI,Legitimidade Judicial,Registro Veterinário,Cobrança de ICMS DIFAL,Competência Juizados Especiais,Acidentes de Trânsito,Classificação de Segurados,Problemas com Operadoras de Telefonia,Impenhorabilidade de Bens Profissionais,Legitimidade em Execuções,Trabalho Externo em Regime Semiaberto,Enquadramento Judicial,Redução de Alimentos,Legitimidade nos Juizados Especiais,Execução de Títulos,Prisão em Processos,Impenhorabilidade de Valores,Competência Juizado Especial,Pagamentos de Serviços,Cobrança sem Nota Fiscal,Legitimidade no Juizado Especial,Benefícios Assistenciais,Aposentadoria Rural,Gratuidade Financeira,Cobrança Indevida,Dívida e Conciliação,Citação Válida,Embargos de Declaração,Crimes e Provas,Justiça Gratuita para Recursos,Doações Eleitorais
Vínculo Empregatício,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Justiça Gratuita,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
Isenção de Taxas para MEIs,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Fraude Empresarial,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Adeção ao Simples Nacional,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Seguro-Desemprego e Renda,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Problemas Bancários,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Confusão Patrimonial,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Negativação Indevida,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Contribuições e Aposentadoria,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


## AGRUPAMENTO: Realiza a busca de grupos no grafo correspondente a matriz
### 1 cria o grafo (com opção de exigir reciprocidade ou não),
### 2 acha os grupos por busca em largura (BFS), Agrupa apenas se o elemento a ser agrupado possui similaridade de até um limiar com os demais
### 3 imprime os grupos em ordem decrescente de tamanho,


In [9]:

def grupos_estritos_por_binaria_e_similaridade(
    df_bin: pd.DataFrame,
    sim_df: pd.DataFrame,
    thr: float = 0.70,                 # exige > thr com TODOS do grupo
    exigir_mutual: bool = False,       # True: precisa A[i,j]==1 e A[j,i]==1
    incluir_isolados: bool = True,     # mantém tópicos sem conexões como grupos unitários
    df_top: Optional[pd.DataFrame] = None  # para escolher "líder" pelo total_registros
):
    # --- 1) Preparar matriz binária e simétrica conforme regra ---
    labels = list(df_bin.index)
    assert labels == list(df_bin.columns) == list(sim_df.index) == list(sim_df.columns), \
        "df_bin e sim_df precisam ter a MESMA ordem de linhas/colunas"

    A = df_bin.applymap(lambda x: 1 if str(x).strip().startswith("1") else 0).values.astype(np.int8)
    if exigir_mutual:
        A_und = (A & A.T).astype(np.int8)
    else:
        A_und = (A | A.T).astype(np.int8)

    # similaridade
    S = sim_df.values.astype(float)
    np.fill_diagonal(A_und, 1)  # um nó conecta a si mesmo

    n = len(labels)
    usados = np.zeros(n, dtype=bool)

    # grau (nº de conexões válidas) para ordenar seeds
    graus = A_und.sum(axis=1)
    seeds = np.argsort(-graus)  # decrescente

    grupos = []
    for s in seeds:
        if usados[s]:
            continue

        # se não quer incluir isolados e s não tem vizinhos, pule
        tem_viz = (A_und[s].sum() - 1) > 0  # desconsidera auto-conexão
        if not incluir_isolados and not tem_viz:
            usados[s] = True
            continue

        # inicia grupo com seed s
        grupo = [s]
        usados[s] = True

        # candidatos elegíveis: conectados ao seed e ainda não usados
        cand = [i for i in range(n) if (i != s) and (not usados[i]) and (A_und[s, i] == 1)]

        # estratégia gulosa: só entra se respeitar TODAS as arestas e similaridade > thr com TODOS do grupo
        # repete varrendo enquanto conseguir incluir alguém
        mudou = True
        while mudou:
            mudou = False
            novos = []
            for i in cand:
                if usados[i]:
                    continue
                # precisa aresta binária com TODOS do grupo...
                if not np.all(A_und[i, grupo] == 1):
                    continue
                # ...e similaridade > thr com TODOS do grupo
                if not np.all(S[i, grupo] > thr):
                    continue
                novos.append(i)

            if novos:
                # para estabilidade, adicione em ordem por (grau decrescente, label)
                novos.sort(key=lambda j: (-graus[j], labels[j].lower()))
                for j in novos:
                    if not usados[j]:
                        grupo.append(j)
                        usados[j] = True
                        mudou = True

                # atualiza candidatos: vizinhos de qualquer membro do grupo, não usados
                vizinhos = np.where(A_und[grupo].any(axis=0))[0]
                cand = [i for i in vizinhos if (not usados[i]) and (i not in grupo)]

        # ordena rótulos do grupo
        grupo_sorted = sorted(grupo, key=lambda idx: labels[idx].lower())
        grupos.append(grupo_sorted)

    # ordena grupos por tamanho e pelo primeiro rótulo
    grupos.sort(key=lambda g: (-len(g), [labels[i].lower() for i in g]))

    # montar saída legível e líder opcional
    linhas, grupos_legiveis = [], []
    for gid, idxs in enumerate(grupos, start=1):
        rotulos = [labels[i] for i in idxs]
        lider = rotulos[0]
        if df_top is not None and {"descricao_topico_llm", "total_registros"}.issubset(df_top.columns):
            sub = df_top[df_top["descricao_topico_llm"].astype(str).isin(rotulos)]
            if not sub.empty:
                lider = sub.sort_values("total_registros", ascending=False).iloc[0]["descricao_topico_llm"]
        grupos_legiveis.append(rotulos)
        for r in rotulos:
            linhas.append({
                "group_id": gid,
                "leader": lider,
                "topic": r,
                "group_size": len(rotulos)
            })

    df_grupos = pd.DataFrame(linhas, columns=["group_id", "leader", "topic", "group_size"])
    return grupos_legiveis, df_grupos




In [10]:
grupos, df_groups = grupos_estritos_por_binaria_e_similaridade(
    df_bin=df_bin,
    sim_df=sim_df,
    thr=0.75,        # seu limiar (ex.: 0.9)
    exigir_mutual=False  # se quiser mais conservador, troque para True
)

for i, g in enumerate(grupos, 1):
    print(f"Grupo {i} ({len(g)}):")
    for t in g:
        print("  -", t)
    print()

# opcional: salvar
df_groups.to_csv("grupos_conectividade.csv", index=False, encoding="utf-8")


Grupo 1 (6):
  - Competência Juizado Especial
  - Competência Juizados Especiais
  - Legitimidade em Execuções
  - Legitimidade Judicial
  - Legitimidade no Juizado Especial
  - Legitimidade nos Juizados Especiais

Grupo 2 (4):
  - Enquadramento Judicial
  - Gratuidade Financeira
  - Justiça Gratuita
  - Justiça Gratuita para Recursos

Grupo 3 (3):
  - Aposentadoria Rural
  - Classificação de Segurados
  - Pensão por Morte

Grupo 4 (2):
  - Benefícios Assistenciais
  - Benefícios para Deficientes

Grupo 5 (2):
  - Benefícios Previdenciários
  - Salário-Maternidade INSS

Grupo 6 (2):
  - Cobrança Indevida
  - Isenção de Taxas para MEIs

Grupo 7 (2):
  - Pagamentos de Serviços
  - Problemas Bancários

Grupo 8 (1):
  - Acidentes de Trânsito

Grupo 9 (1):
  - Adeção ao Simples Nacional

Grupo 10 (1):
  - Auxílio Emergencial

Grupo 11 (1):
  - Citação Válida

Grupo 12 (1):
  - Cobrança de Débitos

Grupo 13 (1):
  - Cobrança de Dívidas Escolares

Grupo 14 (1):
  - Cobrança de ICMS DIFAL

Gru

# Atualizar os integrantes do grupo, escolhendo o representante como o que tem maior qtd de registros

In [11]:
# --- helper: escolhe o representante (maior COUNT(*)) dentro de um grupo de tópicos ---
def escolher_representante(conn, topics):
    """
    Retorna um dict do representante do grupo:
    {descricao_topico_llm, numero_topico_llm, cor_cluster, total}
    """
    with conn.cursor() as cur:
        cur.execute("""
            SELECT 
                descricao_topico_llm,
                numero_topico_llm,
                cor_cluster,
                COUNT(*) AS total
            FROM processos
            WHERE descricao_topico_llm = ANY(%s)
            GROUP BY descricao_topico_llm, numero_topico_llm, cor_cluster
            ORDER BY total DESC
            LIMIT 1
        """, (topics,))
        row = cur.fetchone()
        if not row:
            return None
        return {
            "descricao_topico_llm": row[0],
            "numero_topico_llm": row[1],
            "cor_cluster": row[2],
            "total": row[3],
        }

# --- helper: atualiza todo o grupo para o representante ---
def atualizar_grupo_para_representante(conn, topics, rep):
    """
    Atualiza todos os registros cujos tópicos estejam em 'topics'
    para os campos do representante 'rep'.
    """
    with conn.cursor() as cur:
        cur.execute("""
            UPDATE processos
            SET
                descricao_topico_llm = %s,
                numero_topico_llm   = %s,
                cor_cluster         = %s
            WHERE descricao_topico_llm = ANY(%s)
        """, (
            rep["descricao_topico_llm"],
            rep["numero_topico_llm"],
            rep["cor_cluster"],
            topics
        ))

# --- pipeline principal ---
def aplicar_grupos_por_conectividade(conn, grupos):
    """
    Para cada grupo (lista de títulos de tópico):
      1) escolhe o representante (maior COUNT(*));
      2) atualiza todos os registros dos tópicos do grupo para os campos do representante.
    Tudo numa única transação.
    """
    with conn:
        with conn.cursor() as cur:
            pass  # só para abrir a transação via context manager

        for gid, topics in enumerate(grupos, start=1):
            # sanitiza lista: remove vazios/duplicados
            topics = sorted({str(t).strip() for t in topics if t and str(t).strip()})
            if not topics:
                continue

            rep = escolher_representante(conn, topics)
            if not rep:
                print(f"[Grupo {gid}] Sem registros para: {topics}")
                continue

            atualizar_grupo_para_representante(conn, topics, rep)
            print(f"[Grupo {gid}] {topics}  →  representante: "
                  f"{rep['descricao_topico_llm']} (nº {rep['numero_topico_llm']}, cor {rep['cor_cluster']}, total={rep['total']})")

    print("✅ Atualizações concluídas.")

conn = get_connection()
aplicar_grupos_por_conectividade(conn, grupos)


[Grupo 1] ['Competência Juizado Especial', 'Competência Juizados Especiais', 'Legitimidade Judicial', 'Legitimidade em Execuções', 'Legitimidade no Juizado Especial', 'Legitimidade nos Juizados Especiais']  →  representante: Legitimidade Judicial (nº 43, cor #875692, total=51)
[Grupo 2] ['Enquadramento Judicial', 'Gratuidade Financeira', 'Justiça Gratuita', 'Justiça Gratuita para Recursos']  →  representante: Justiça Gratuita (nº 1, cor #55B4F6, total=999)
[Grupo 3] ['Aposentadoria Rural', 'Classificação de Segurados', 'Pensão por Morte']  →  representante: Pensão por Morte (nº 28, cor #6419B2, total=58)
[Grupo 4] ['Benefícios Assistenciais', 'Benefícios para Deficientes']  →  representante: Benefícios para Deficientes (nº 46, cor #F655DC, total=74)
[Grupo 5] ['Benefícios Previdenciários', 'Salário-Maternidade INSS']  →  representante: Benefícios Previdenciários (nº 37, cor #E86762, total=179)
[Grupo 6] ['Cobrança Indevida', 'Isenção de Taxas para MEIs']  →  representante: Isenção de T

In [12]:
# Obtendo novamente a descrição e titulo de cada cluster, pois agora foram agrupados e é interessante atualizar

In [13]:
def gerar_e_salvar_detalhes_cluster(modelo, token):
    conn = get_connection()
    cur = conn.cursor()
    modelo = 'pixtral-12b'

    # =========================
    # 0) Limpar campos antigos antes de atualizar
    # =========================
    print("🧹 Limpando campos anteriores dos clusters...")
    cur.execute("""
        UPDATE processos SET
            descricao_curta_cluster = NULL,
            descricao_longa_cluster = NULL,
            questoes_discussao_cluster = NULL,
            solucoes_propostas_cluster = NULL,
            teses_cluster = NULL
    """)
    conn.commit()
    print("✅ Campos limpos com sucesso.\n")

    df = pd.read_sql_query("""
        SELECT id, numero_processo_tribunal, descricao_caso, questoes_em_discussao, 
               solucoes_propostas, tese, numero_topico_llm, descricao_topico_llm, 
               proximo_do_centroid, descricao_topico_bertopic_padrao
        FROM processos 
        WHERE proximo_do_centroid = 1
    """, conn)

    grupos = df.groupby("descricao_topico_llm")

    for topico, grupo in grupos:
        casos = "\n\n".join(grupo['descricao_caso'].dropna())
        keywords = "\n\n Palavras chave:".join(grupo['descricao_caso'].dropna())
        questoes = "\n\n".join(grupo['questoes_em_discussao'].dropna())
        solucoes = "\n\n".join(grupo['solucoes_propostas'].dropna())
        teses = "\n\n".join(grupo['tese'].dropna())

        clear_output(wait=True)
        print(f"\n\n===========================")
        print(f"🔎 Tópico {topico} - {grupo['descricao_topico_llm'].iloc[0]}")
        print(f"IDs: {grupo['id'].tolist()}")

        # 0. Título do cluster
        prompt_0 = """Você é um assistente inteligente de extração de tópicos, especializado em nomear tópicos de forma curta, clara e amigável, com base em textos representativos e palavras-chave. 
        Seu objetivo é criar um **rótulo conciso** que represente o problema central em comum dos conteúdos discutidos no tópico, facilitando a identificação do assunto por usuários finais. 
        
        Siga as diretrizes abaixo:
        - Não inclua o termo "MEI" no nome do tópico.
        - O nome deve ter de 1 a 3 palavras.
        - Não cite nome de pessoas, locais ou instituições.

        Responda apenas o nome do rótulo, sem asteriscos.
        """ + keywords + casos
        texto_0 = invoke(prompt_0, modelo, token)
        print("\n📌 Título:")
        print(texto_0)

        # 1. Descrição curta
        prompt_1 = "Você é um assistente jurídico. Com base nas descrições dos casos abaixo, escreva um resumo curto (1 parágrafo com até 150 caracteres) que represente o tema comum entre os processos:\n\n" + casos
        #print(prompt_1)
        texto_1 = invoke(prompt_1, modelo, token)
        
        print("\n📌 Descrição curta:")
        print(texto_1)

        # 2. Descrição longa
        prompt_2 = "Você é um assistente jurídico. Com base nas descrições dos casos abaixo, escreva um texto descritivo em até 500 caracteres sobre o tema comum desses processos:\n\n" + casos
        texto_2 = invoke(prompt_2, modelo, token)
        print("\n📘 Descrição longa:")
        print(texto_2)
        
        # 3. Questões em discussão
        prompt_3 = "A partir das informações abaixo, escreva até 5 principais questões comuns em discussão nos processos, o retorno não precisa ser em formato de pergunta:\n\n" + questoes
        texto_3 = invoke(prompt_3, modelo, token)
        print("\n❓ Questões em discussão:")
        print(texto_3)


        # 4. Soluções propostas
        prompt_4 = "A partir das informações abaixo, escreva um resumo em até 500 caracteres das soluções propostas comuns nos processos:\n\n" + solucoes
        texto_4 = invoke(prompt_4, modelo, token)
        print("\n💡 Soluções propostas:")
        print(texto_4)
        texto_4 = ""
        
        # 5. Teses jurídicas
        prompt_5 = "A partir das informações abaixo, escreva um resumo em até 500 caracteres das teses jurídicas comuns nesses processos:\n\n" + teses
        texto_5 = invoke(prompt_5, modelo, token)
        print("\n⚖️ Teses jurídicas:")
        print(texto_5)
        texto_5 = ""
        
        # Atualiza apenas uma vez por topico LLM
        cur.execute("""
            UPDATE processos SET 
                descricao_topico_llm = %s,
                descricao_curta_cluster = %s,
                descricao_longa_cluster = %s,
                questoes_discussao_cluster = %s,
                solucoes_propostas_cluster = %s,
                teses_cluster = %s
            WHERE descricao_topico_llm = %s
        """, (
            texto_0, texto_1, texto_2, texto_3, texto_4, texto_5, topico
        ))
        conn.commit()

    cur.close()
    conn.close()


In [14]:
gerar_e_salvar_detalhes_cluster("pixtral-12b", get_token())



🔎 Tópico Vínculo Empregatício - Vínculo Empregatício
IDs: [8373, 6454, 5750, 32061, 11256, 43466, 14912, 11443, 31896, 35651, 10501, 30897, 9463, 25338, 36301, 45160, 9333, 42309, 35076, 27675, 36413, 11209, 16386, 13719, 6630, 26108, 7099, 7976, 11210, 23714, 7100, 24618, 25583, 5967, 7385, 42775, 12463, 18253, 44825, 7179, 8765, 5698, 14611, 28501, 8396, 10614, 6132, 6178, 35582, 8433, 35077, 12491, 42650, 36045, 12902, 15219, 44500, 8525, 26250, 43173, 6415, 10769, 5985, 14344, 7386, 9191, 24780, 10750, 16660, 23204, 43025, 45020, 7259, 36691, 24737, 23943, 17374, 36290, 35377, 36773, 35235, 16946, 44081, 16731, 36452, 42956, 43659, 17141, 9139, 44397, 25271, 6406, 13210, 8247, 44499, 17971, 23467, 24434, 36541, 36312, 7826, 27593, 43748, 14970, 16360, 27364, 27363, 27188, 8274, 8384, 43903, 26113, 35459, 15885, 23838, 8301, 28085, 14089, 35858, 8679, 17686, 14323, 27824, 42955, 7895, 18046, 27246, 8313, 28298, 8257, 36388, 18598, 24832, 5973, 27030, 12327, 4808, 26498, 42615, 247

In [15]:
def traduzir_nome_cluster(modelo, token):
    conn = get_connection()
    cur = conn.cursor()
    modelo = 'pixtral-12b'

    df = pd.read_sql_query("""
        SELECT numero_topico_llm, descricao_topico_llm, descricao_curta_cluster
        FROM processos 
        WHERE proximo_do_centroid = 1
    """, conn)

    grupos = df.groupby("descricao_topico_llm")

    for topico, grupo in grupos:
        titulo = "\n\nNome do Tópico:".join(grupo['descricao_topico_llm'].dropna())
        descricao = "\n\nDescrição do Tópico:".join(grupo['descricao_curta_cluster'].dropna())
        clear_output(wait=True)
        print(f"\n\n===========================")
        print(f"🔎 Tópico {topico} - {grupo['descricao_topico_llm'].iloc[0]}")

        # 0. Título do cluster
        prompt_0 = """Traduza para Inglês o nome do tópico abaixo. Lembre que são termos jurídicos. Responda apenas o nome do tópico em inglês , sem asteriscos.
        """ + titulo + descricao
        texto_0 = invoke(prompt_0, modelo, token)
        print("\n📌 Título:")
        print(texto_0)
        # Atualiza apenas uma vez por topico LLM
        cur.execute("""
            UPDATE processos SET 
                descricao_topico_llm_ingles = %s
            WHERE descricao_topico_llm = %s
        """, (
            texto_0, topico
        ))
        conn.commit()

    cur.close()
    conn.close()


In [16]:
traduzir_nome_cluster("pixtral-12b", get_token())



🔎 Tópico Vínculo Empregatício - Vínculo Empregatício

📌 Título:
Employment Relationship
